# Frozen Lake (Q-Learning)

Contoh Deloyment untuk Domain Reinforcement Learning (RL) <br>
Orbit Future Academy - AI Mastery - KM Batch 3 <br>
Tim Deployment dan Tim RL <br>
2022

## Modules dan Packages

In [ ]:
!pip install gym==0.17.3 --quiet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.3 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654650 sha256=1dfc463b77cee8d87ba9ea65555d3964e739a6dd703d0df06d9f1f623db2c11c
  Stored in directory: /root/.cache/pip/wheels/af/4b/74/fcfc8238472c34d7f96508a63c962ff3ac9485a9a4137afd4e
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [ ]:
import numpy as np
import gym
import pickle

## Load Environment

In [ ]:
# Peta (ada 5 peta)
peta = [
    ['SFFF','FHFH','FFFH','HFFG'],
    ['SFFF','FFHF','HFFF','HFFG'],
    ['SHFF','FHFH','FFFH','HHFG'],
    ['SFFF','HHFF','FFFF','HFFG'],
    ['SFFH','FFFH','HFFH','HHFG']
]

In [ ]:
# Load Environment
env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta[0])

In [ ]:
n_observations = env.observation_space.n
n_actions      = env.action_space.n

print('Banyak State  : ' + str(n_observations))
print('Banyak Action : ' + str(n_actions))

Banyak State  : 16
Banyak Action : 4


In [ ]:
ACTION = ["KIRI","BAWAH","KANAN","ATAS"]

In [ ]:
env.reset()
env.render()


SFFF
FHFH
FFFH
HFFG


In [ ]:
# Langkah 1 (ke Kanan)
new_state, reward, done, info = env.step(2)

print('New State : ' + str(new_state))
print('Reward    : ' + str(reward))
print('Done      : ' + str(done))

env.render()

New State : 1
Reward    : 0.0
Done      : False
  (Right)
SFFF
FHFH
FFFH
HFFG


In [ ]:
# Langkah 2 (ke Kanan)
new_state, reward, done, info = env.step(2)

print('New State : ' + str(new_state))
print('Reward    : ' + str(reward))
print('Done      : ' + str(done))

env.render()

New State : 2
Reward    : 0.0
Done      : False
  (Right)
SFFF
FHFH
FFFH
HFFG


In [ ]:
# Langkah 3 (ke Bawah)
new_state, reward, done, info = env.step(1)

print('New State : ' + str(new_state))
print('Reward    : ' + str(reward))
print('Done      : ' + str(done))

env.render()

New State : 6
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


In [ ]:
# Langkah 4 (ke Bawah)
new_state, reward, done, info = env.step(1)

print('New State : ' + str(new_state))
print('Reward    : ' + str(reward))
print('Done      : ' + str(done))

env.render()

New State : 10
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


In [ ]:
# Langkah 5 (ke Bawah)
new_state, reward, done, info = env.step(1)

print('New State : ' + str(new_state))
print('Reward    : ' + str(reward))
print('Done      : ' + str(done))

env.render()

New State : 14
Reward    : 0.0
Done      : False
  (Down)
SFFF
FHFH
FFFH
HFFG


In [ ]:
# Langkah 6 (ke Kanan)
new_state, reward, done, info = env.step(2)

print('New State : ' + str(new_state))
print('Reward    : ' + str(reward))
print('Done      : ' + str(done))

env.render()

New State : 15
Reward    : 1.0
Done      : True
  (Right)
SFFF
FHFH
FFFH
HFFG


## Training Model/Agent

In [ ]:
def train_agent(env):
    # banyak episode yang akan dijalankan
    n_episodes = 10000

    # iterasi maksimum per episode
    max_iter_episode = 100

    # inisialisasi probabilitas eksplorasi
    exploration_proba = 1

    # penurunan eksplorasi secara eksponensial
    exploration_decreasing_decay = 0.001

    # probabilitas eksplorasi minimum
    min_exploration_proba = 0.01

    # faktor diskon
    gamma = 0.99

    # learning rate
    lr = 0.1
    
    # Inisialisasi Q-table 
    Q_table = np.zeros((n_observations,n_actions))
    
    total_rewards_episode = list()
    rewards_per_episode=[]
    
    for e in range(n_episodes):
        current_state = env.reset()
        done = False
        total_episode_reward = 0

        for i in range(max_iter_episode):      
            if np.random.uniform(0,1) < exploration_proba:
                action = env.action_space.sample()
            else:
                action = np.argmax(Q_table[current_state,:])

            next_state, reward, done, _ = env.step(action)

            Q_table[current_state, action] = (1-lr) * Q_table[current_state, action] +lr*(reward + gamma*max(Q_table[next_state,:]))
            total_episode_reward = total_episode_reward + reward

            if done:
                break

            current_state = next_state

        exploration_proba = max(min_exploration_proba, np.exp(-exploration_decreasing_decay*e))
        rewards_per_episode.append(total_episode_reward)
    
    print("Rata-Rata Reward per 1000 Episode")
    for i in range(10):
        print((i+1)*1000, " : Rata-Rata Reward: ", np.mean(rewards_per_episode[1000*i:1000*(i+1)])) 
    
    return Q_table

In [ ]:
Q_table_all = []

for peta_env in peta:
    # Load Environment untuk setiap peta
    env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta_env)
    env.reset()
    
    print('Peta : ')
    print(peta_env)
    print()
    
    # Melatih Agent
    Q_table = train_agent(env)
    
    # Menyimpan Q_table
    Q_table_all.append(Q_table)
    
    print()

Peta : 
['SFFF', 'FHFH', 'FFFH', 'HFFG']

Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.255
2000  : Rata-Rata Reward:  0.751
3000  : Rata-Rata Reward:  0.925
4000  : Rata-Rata Reward:  0.974
5000  : Rata-Rata Reward:  0.988
6000  : Rata-Rata Reward:  0.989
7000  : Rata-Rata Reward:  0.988
8000  : Rata-Rata Reward:  0.989
9000  : Rata-Rata Reward:  0.987
10000  : Rata-Rata Reward:  0.994

Peta : 
['SFFF', 'FFHF', 'HFFF', 'HFFG']

Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.427
2000  : Rata-Rata Reward:  0.873
3000  : Rata-Rata Reward:  0.96
4000  : Rata-Rata Reward:  0.978
5000  : Rata-Rata Reward:  0.992
6000  : Rata-Rata Reward:  0.994
7000  : Rata-Rata Reward:  0.997
8000  : Rata-Rata Reward:  0.994
9000  : Rata-Rata Reward:  0.995
10000  : Rata-Rata Reward:  0.991

Peta : 
['SHFF', 'FHFH', 'FFFH', 'HHFG']

Rata-Rata Reward per 1000 Episode
1000  : Rata-Rata Reward:  0.003
2000  : Rata-Rata Reward:  0.57
3000  : Rata-Rata Reward:  0.864
4000  : Rat

## Memainkan Agent yang Telah Dilatih

In [ ]:
index_peta = 0 # silahkan pilih peta

env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta[index_peta])
env.reset()

0

In [ ]:
for langkah in range(1,7):
    if langkah == 1:
        best_action = np.argmax(Q_table_all[index_peta][0])
    else:
        best_action = np.argmax(Q_table_all[index_peta][current_state])
        
    new_state, reward, done, info = env.step(best_action)

    print('Langkah ke  : ' + str(langkah))
    print('Best Action : ' + ACTION[best_action])
    print('New State   : ' + str(new_state))
    print('Reward      : ' + str(reward))
    print('Done        : ' + str(done))
    print()

    env.render()
    
    current_state = new_state
    
    print()

Langkah ke  : 1
Best Action : KANAN
New State   : 1
Reward      : 0.0
Done        : False

  (Right)
SFFF
FHFH
FFFH
HFFG

Langkah ke  : 2
Best Action : KANAN
New State   : 2
Reward      : 0.0
Done        : False

  (Right)
SFFF
FHFH
FFFH
HFFG

Langkah ke  : 3
Best Action : BAWAH
New State   : 6
Reward      : 0.0
Done        : False

  (Down)
SFFF
FHFH
FFFH
HFFG

Langkah ke  : 4
Best Action : BAWAH
New State   : 10
Reward      : 0.0
Done        : False

  (Down)
SFFF
FHFH
FFFH
HFFG

Langkah ke  : 5
Best Action : BAWAH
New State   : 14
Reward      : 0.0
Done        : False

  (Down)
SFFF
FHFH
FFFH
HFFG

Langkah ke  : 6
Best Action : KANAN
New State   : 15
Reward      : 1.0
Done        : True

  (Right)
SFFF
FHFH
FFFH
HFFG



## Periksa Apakah Agent Dapat Menyelesaikan Semua Peta

In [ ]:
for peta_env in peta:
    print("Peta   : " + str(peta_env))
    
    env = gym.make("FrozenLake-v0",is_slippery=False, desc=peta_env)
    env.reset()
    
    for langkah in range(1,7):
        if langkah == 1:
            best_action = np.argmax(Q_table_all[index_peta][0])
        else:
            best_action = np.argmax(Q_table_all[index_peta][current_state])

        new_state, reward, done, info = env.step(best_action)

        current_state = new_state

    if done == True:
        print("Status : Agent dapat menyelesaikan peta ini")
    else:
        print("Status : Agent tidak dapat menyelesaikan peta ini")
        
    print()

Peta   : ['SFFF', 'FHFH', 'FFFH', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFF', 'FFHF', 'HFFF', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SHFF', 'FHFH', 'FFFH', 'HHFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFF', 'HHFF', 'FFFF', 'HFFG']
Status : Agent dapat menyelesaikan peta ini

Peta   : ['SFFH', 'FFFH', 'HFFH', 'HHFG']
Status : Agent dapat menyelesaikan peta ini



## Menyimpan Model

In [ ]:
pickle.dump(Q_table_all, open('Q_table_Frozen_Lake.model', 'wb'))